In [33]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import time 
import os 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler


In [15]:
data=pd.read_csv('/home/jabed/Downloads/Desktop_backup/Desk1/ML_tensor_pytorch/MultilabelClassification/Tensorlfow/IRIS.csv')

#data=pd.get_dummies(data,columns=['species'])
df=data
species = dict(zip(list(df['species'].unique()), ([0, 1, 2])))
df['species'].replace(species,inplace=True)


In [40]:
X=df.drop('species',axis=1)
Y=df['species'].values
y_dummy=pd.get_dummies(Y)
X=X.values


In [45]:
X=np.array(X,dtype=np.float32)

In [50]:
class Linear(tf.keras.layers.Layer):
    def __init__(self,units=1):
        super(Linear,self).__init__()
        self.units=units
    def build(self,input_shape):
        self.weight=self.add_weight(
            shape=(input_shape[-1],self.units),
            initializer='random_normal',
            trainable=True,

        )    
        self.bias = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )
    def call(self,inputs):
        return tf.matmul(inputs,self.weight)+self.bias 
    def get_config(self):
        return {'units':self.units}

In [66]:
class MultiLabel(tf.keras.Model):
    def __init__(self,num_class):
        super(MultiLabel,self).__init__()


        self.linear=Linear(num_class)
        
    def call(self,inputs):
        l1=self.linear(inputs)
        out=tf.nn.sigmoid(l1)
        return out 
    def accuracy(self,y_dummy,y_pred):
        acc=tf.keras.metrics.Accuracy()
        acc.update_state(y_dummy,tf.argmax(y_pred,axis=1))    
        return acc.result().numpy()
        



In [67]:
model=MultiLabel(3)

In [77]:
y_pred=model(X)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [84]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
logits=model.call(X)
loss_value=loss_fn(y_dummy,logits)

In [87]:
def fit(X_train,y_train,num_epochs=100,lr=0.01,y_dummy=y_dummy):
    start=time.time()

    j_history=[]
    loss_fn=tf.keras.losses.CategoricalCrossentropy()

    optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
    for epoch in range(num_epochs+1):
        with tf.GradientTape() as tape:
            logits=model.call(X_train)
            loss_value=loss_fn(y_dummy,logits)


        grads=tape.gradient(loss_value,model.trainable_weights)
        optimizer.apply_gradients(zip(grads,model.trainable_weights))
        acc=model.accuracy(y_train,logits)
        j_history.append((loss_value.numpy(),acc))

        if epoch %10==0:
            print('epoch {} loss {:.4f} and accuracy {:.4f}'.format(epoch,loss_value.numpy(),acc))   
            #print(tf.argmax(logits[0:20],axis=1))  

    end=time.time()
    print('totla time = ',end-start)
    return np.array(j_history)

In [88]:
j_history=fit(X_train=X,y_train=Y,num_epochs=100,lr=0.01,y_dummy=y_dummy)

epoch 0 loss 1.0628 and accuracy 0.6333
epoch 10 loss 0.9413 and accuracy 0.6600
epoch 20 loss 0.8104 and accuracy 0.6667
epoch 30 loss 0.6983 and accuracy 0.6667
epoch 40 loss 0.6145 and accuracy 0.6667
epoch 50 loss 0.5523 and accuracy 0.7533
epoch 60 loss 0.5047 and accuracy 0.8667
epoch 70 loss 0.4670 and accuracy 0.9267
epoch 80 loss 0.4361 and accuracy 0.9400
epoch 90 loss 0.4100 and accuracy 0.9533
epoch 100 loss 0.3875 and accuracy 0.9667
totla time =  2.7951791286468506


array([[1.0628146 , 0.6333333 ],
       [1.0501735 , 0.55333334],
       [1.0384269 , 0.45333335],
       [1.0270599 , 0.38      ],
       [1.0155518 , 0.39333335],
       [1.0037714 , 0.44666666],
       [0.9917168 , 0.52      ],
       [0.97941   , 0.56666666],
       [0.96687865, 0.64666665],
       [0.954151  , 0.66      ],
       [0.94125456, 0.66      ],
       [0.92821836, 0.66      ],
       [0.9150729 , 0.6666667 ],
       [0.90185076, 0.6666667 ],
       [0.88858783, 0.6666667 ],
       [0.8753223 , 0.6666667 ],
       [0.862094  , 0.6666667 ],
       [0.848944  , 0.6666667 ],
       [0.8359121 , 0.6666667 ],
       [0.8230367 , 0.6666667 ],
       [0.8103517 , 0.6666667 ],
       [0.79788744, 0.6666667 ],
       [0.7856686 , 0.6666667 ],
       [0.7737152 , 0.6666667 ],
       [0.762042  , 0.6666667 ],
       [0.75065947, 0.6666667 ],
       [0.7395746 , 0.6666667 ],
       [0.7287912 , 0.6666667 ],
       [0.7183109 , 0.6666667 ],
       [0.7081337 , 0.6666667 ],
       [0.